In [0]:
%run "/Workspace/Repos/ashwin.nair8089@gmail.com/inshorts/library/connector"

In [0]:
%run "/Workspace/Repos/ashwin.nair8089@gmail.com/inshorts/library/password_manager"

In [0]:
conn, success, message = snowflake_connector("ashwin009", get_password('snf_conn'), "WSYCJNH-JF51761",'COMPUTE_WH','INSHORTS','ETL')

In [0]:
cursor = conn.cursor()

In [0]:

cursor.execute('Truncate table foundation.monthly_churn')
print(cursor.fetchall())
cursor.execute(f'''INSERT INTO foundation.monthly_churn
SELECT 
    u.install_dt, 
    FLOOR(DATEDIFF(DAY, u.install_dt, e.event_date) / 30) + 1 AS month_number,
    COUNT(DISTINCT u.deviceid) - COUNT(DISTINCT e.deviceid) AS churned_users,
    (COUNT(DISTINCT u.deviceid) - COUNT(DISTINCT e.deviceid)) / COUNT(DISTINCT u.deviceid) AS churn_rate
FROM ETL.user u
LEFT JOIN (
    SELECT deviceid, DATE(eventtimestamp) AS event_date
    FROM ETL.event    
) e ON u.deviceid = e.deviceid
WHERE e.event_date BETWEEN u.install_dt AND DATEADD(MONTH, 3, u.install_dt) -- Covers first quarter
GROUP BY u.install_dt, month_number;''')
print(cursor.fetchall())